In [1]:
import pandas as pd
from tqdm import tqdm
import torch
import pytorch_utils
DATA_DIR = 'data/'

[PyTorch-Utils]: Loading anchor candidate data...
[PyTorch-Utils]: Loading item_id to statement embedding data...
[PyTorch-Utils]: Loading item_id to description embedding data...
[PyTorch-Utils]: Loading wikipedia title embedings...
[PyTorch-Utils]: Loading wikipedia items...


In [2]:
# is cuda available?
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = pytorch_utils.EntityDataset(device='cuda')
test_dataset = pytorch_utils.EntityDataset(train=False, device='cuda')

[PyTorch-Utils]: Loading train set...
[PyTorch-Utils]: Now generating context embeddings...
[PyTorch-Utils]: Loading train_context_text_150.pkl from data/pkl...


Batches:   0%|          | 0/572 [00:00<?, ?it/s]

[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/572 [00:00<?, ?it/s]

Entity Length: 18288
Entity shape: torch.Size([18288, 384])
[PyTorch-Utils]: Now calculating syntactic neighbours...
[PyTorch-Utils]: Loading test set...
[PyTorch-Utils]: Now generating context embeddings...
[PyTorch-Utils]: Loading test_context_text_150.pkl from data/pkl...


Batches:   0%|          | 0/287 [00:00<?, ?it/s]

[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/287 [00:00<?, ?it/s]

Entity Length: 9166
Entity shape: torch.Size([9166, 384])
[PyTorch-Utils]: Now calculating syntactic neighbours...


In [4]:
pytorch_utils.delete_corpus_embeds()

[PyTorch-Utils]: Deleting corpus embeddings...


In [5]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_train(x, device=device, entity_embeddings=dataset.entity_embeddings,context_embeddings=dataset.context_embeddings,syntax_candidates_list=dataset.syntax_candidates))
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn_test(x, device=device, entity_embeddings=test_dataset.entity_embeddings,context_embeddings=test_dataset.context_embeddings,syntax_candidates_list=test_dataset.syntax_candidates))

In [6]:
model = pytorch_utils.EntityClassifier(device=device)

In [7]:
# loss and optimizer
from torch import optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
from tqdm import tqdm


epochs = 20
pbar = tqdm(range(epochs))
model.train()
for epoch in pbar:
    running_loss = 0.0
    shapes = []
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch, labels_batch = data
        shapes.append(batch_context_embeddings.shape[0])
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model((batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch))
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    pbar.set_description(f'Epoch {epoch} loss: {running_loss / len(dataloader)} Avg shape: {sum(shapes) / len(shapes)}')

  0%|          | 0/20 [00:00<?, ?it/s]

c:\Users\aybar\Documents\CS423-Project-3\pytorch_utils.py:226: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  candidate_description_embeddings_batch = torch.tensor(candidate_description_embeddings_batch, dtype=torch.float32, device=device)
Epoch 19 loss: 0.03856936628646152 Avg shape: 54.19580419580419: 100%|██████████| 20/20 [02:05<00:00,  6.28s/it] 


In [9]:
# let's do predictions on the test set
model.eval()
with torch.no_grad():
    predictions = []
    prediction_indexes = []
    for i, data in tqdm(enumerate(test_dataloader)):
        batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch = data
        outputs = model((batch_context_embeddings, batch_entity_embeddings, candidate_ids_batch, candidate_description_embeddings_batch))
        # which index has the highest value?
        _, predicted = torch.max(outputs, 1)
        prediction_indexes.extend(predicted.tolist())
        # predicted is shape (batch_size)
        # we want to grab the best candidate for batch item i
        # i.e the candidate_ids is of shape (batch_size, 5)
        # we want to grab the best candidate for batch item i
        # i.e get it to shape (batch_size)
        best_candidates = candidate_ids_batch[torch.arange(candidate_ids_batch.size(0)), predicted]

        # Append the best candidates to the predictions list
        predictions.extend(best_candidates.tolist())
        

144it [00:03, 47.73it/s]


In [10]:
# predictions_from_statistics = []

# for i, row in tqdm(test_dataset.entity_df.iterrows()):
#     full_mention = row['full_mention'].strip().lower()
#     if full_mention in pytorch_utils.anchor_to_candidate:
#         best_candidate = pytorch_utils.anchor_to_candidate[full_mention][0]
#     else:
#         best_candidate = 0
#     predictions_from_statistics.append(best_candidate)

In [11]:
wiki_items = pd.read_csv(DATA_DIR + 'wiki_lite/wiki_items.csv')
# index wiki_items by id
wiki_items = wiki_items.set_index('item_id')
# Create item_id to wikipedia_title map
item_id_to_title = wiki_items['wikipedia_title'].to_dict()

In [12]:
enwiki_redirects = pd.read_csv(DATA_DIR + 'wiki_lite/enwiki_redirects.tsv', sep='\t', header=None, names=['source', 'target'])
# index enwiki_redirects by source
enwiki_redirects = enwiki_redirects.set_index('source')
# create source to target map
source_to_target = enwiki_redirects['target'].to_dict()

In [13]:
wiki_urls = []
not_found = 0
found = 0
# Now we will map these into wikipedia_urls
for i in tqdm(range(len(predictions))):
    if predictions[i] == 0:
        # if the prediction is 0, we will append a blank url
        wiki_urls.append('NOT_FOUND')
        not_found += 1
        continue
    wikipedia_title = item_id_to_title[predictions[i]]
    # does this wikipedia title exist in the redirects?
    if wikipedia_title in source_to_target:
        # if it does, we will replace it with the redirect
        new_title = source_to_target[wikipedia_title]
        wikipedia_title = new_title
    # Now replace the spaces with underscores
    wikipedia_title = wikipedia_title.replace(' ', '_')
    # And add the wikipedia url
    wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')
    found += 1

print(f'Found {found} wikipedia urls')
print(f'Not found {not_found} wikipedia urls')
print(f'Percentage of wikipedia urls found: {found / (found + not_found)}')

100%|██████████| 9166/9166 [00:00<00:00, 897157.44it/s]

Found 8803 wikipedia urls
Not found 363 wikipedia urls
Percentage of wikipedia urls found: 0.9603971197905302


In [14]:
# not_found = 0
# found = 0
# train_wiki_urls = []
# # Now we will map these into wikipedia_urls
# for i in tqdm(range(len(predictions_train))):
#     if predictions_train[i] == 0:
#         # if the prediction is 0, we will append a blank url
#         train_wiki_urls.append('NOT_FOUND')
#         not_found += 1
#         continue
#     wikipedia_title = item_id_to_title[predictions_train[i]]
#     # does this wikipedia title exist in the redirects?
#     if wikipedia_title in source_to_target:
#         # if it does, we will replace it with the redirect
#         new_title = source_to_target[wikipedia_title]
#     # Now replace the spaces with underscores
#     wikipedia_title = wikipedia_title.replace(' ', '_')
#     # And add the wikipedia url
#     train_wiki_urls.append(f'http://en.wikipedia.org/wiki/{wikipedia_title}')
#     found += 1

# print(f'Found {found} wikipedia urls')
# print(f'Not found {not_found} wikipedia urls')
# print(f'Percentage of wikipedia urls found: {found / (found + not_found)}')

In [15]:
test = pd.read_csv(DATA_DIR + 'test.csv')
# train = pd.read_csv(DATA_DIR + 'train.csv')

In [16]:
not_nan = test['wiki_url'].notna()
not_nme = test['wiki_url'] != '--NME--'
# train_not_nan = train['wiki_url'].notna()
# train_not_nme = train['wiki_url'] != '--NME--'
test.loc[(not_nan & not_nme) & (test.id == 65002)]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,?


In [17]:
test.loc[not_nan & not_nme, 'wiki_url'] = wiki_urls
# train.loc[train_not_nan & train_not_nme, 'wiki_url'] = train_wiki_urls

In [18]:
# replace NaN or --NME-- with NOT_FOUND
test['wiki_url'] = test['wiki_url'].fillna('NOT_FOUND')
test['wiki_url'] = test['wiki_url'].replace('--NME--', 'NOT_FOUND')
# train['wiki_url'] = train['wiki_url'].fillna('NOT_FOUND')
# train['wiki_url'] = train['wiki_url'].replace('--NME--', 'NOT_FOUND')

In [19]:
test[test.id == 65002]

,id,token,entity_tag,full_mention,wiki_url
65002,65002,Dejan,B,Dejan Koturovic,http://en.wikipedia.org/wiki/Dejan_Koturović


In [20]:
' '.join(test.token.fillna('', inplace=False).to_list()[:250])

"-DOCSTART- (947testa CRICKET) CRICKET - LEICESTERSHIRE TAKE OVER AT TOP AFTER INNINGS VICTORY .  LONDON 1996-08-30  West Indian all-rounder Phil Simmons took four for 38 on Friday as Leicestershire beat Somerset by an innings and 39 runs in two days to take over at the head of the county championship .  Their stay on top , though , may be short-lived as title rivals Essex , Derbyshire and Surrey all closed in on victory while Kent made up for lost time in their rain-affected match against Nottinghamshire .  After bowling Somerset out for 83 on the opening morning at Grace Road , Leicestershire extended their first innings by 94 runs before being bowled out for 296 with England discard Andy Caddick taking three for 83 .  Trailing by 213 , Somerset got a solid start to their second innings before Simmons stepped in to bundle them out for 174 .  Essex , however , look certain to regain their top spot after Nasser Hussain and Peter Such gave them a firm grip on their match against Yorkshi

In [21]:
# # One problem, Simmons does not retrieve the cricket player, but querying for Phil Simmons does.
# # TODO: Fix this, one idea is to group by per doc id, check if this token has a better previous full mention
# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['simmons'])])

# print('____')

# display(wiki_items[wiki_items.index.isin(test_dataset.anchor_to_candidate['phil simmons'])])

In [22]:
# now create a .csv file from id, wiki_url
test[['id', 'wiki_url']].to_csv('submission_after_structure_change.csv', index=False)
# train[['id', 'wiki_url']].to_csv('train_with_doc.csv', index=False)